# API REST - Query from Python

Retrieves data __from the following REST API__.

API documentation is [available here](https://data.nantesmetropole.fr/explore/dataset/244400404_stations-velos-libre-service-nantes-metropole-disponibilites/api/).

Displays a DataFrame with the address of each station, and the associated fill rate (i.e. the number of available bikes `available_bikes` / the number of total slots `bike_stands`).

Expected result (the fill rates will be different, as the data is generated in real time):

![screenshot of expected result](http://images.innoveduc.fr/data/collect/JSON/expected_result_API_biclou.png)

In [3]:
# Import necessary modules
import requests
import json
import pandas as pd

In [45]:
# I decided to store API request config in a dictionary
# so it is easier to change the query parameters.
# Not that I need it now, but a good exercise.
api_config = {
    "endpoint"  : "https://data.nantesmetropole.fr",
    "api"       : "/api/records/1.0",
    "operation" : "/search",
    "parameters": [
        {"name" : "dataset",
         "value": "244400404_stations-velos-libre-service-nantes-metropole-disponibilites"},
        {"name" : "q",
         "value":  ""}]
}

# Function to generate URI from "api_config"
def generate_uri(cfg):
    uri = (cfg["endpoint"]
           + cfg["api"]
           + cfg["operation"]
           + "/?"
           + "&".join([f"{parameter['name']}={parameter['value']}" for parameter in cfg["parameters"]]))
    return uri

# Getting amount of available rows (bike stations) from the API
# Generating initial request URI
uri = generate_uri(api_config)

# Getting response from HTTP GET request
r = requests.get(uri)

# Loading the JSON document
doc = json.loads(r.content)

# Extracting and adding new parameter (rows) to the URI config
api_config["parameters"].append({"name": "rows", "value": str(doc["nhits"])})

# Regenerate URI again (with rows)
uri = generate_uri(api_config)

In [46]:
# Getting response from HTTP GET request
r = requests.get(uri)

# Loading the JSON document
doc = json.loads(r.content)

df = pd.json_normalize(doc, record_path="records")[["fields.address", "fields.available_bikes", "fields.bike_stands"]]
df["fill_rate"] = df["fields.available_bikes"] / df["fields.bike_stands"]
df.drop(columns=["fields.available_bikes", "fields.bike_stands"]).rename(columns={"fields.address":"address"})

,address,fill_rate
0,NaN,0.200000
1,"42, boulevard Gabriel Guist'Hau",0.133333
2,"11, rue de Belfort",0.529412
3,Rond-Point de Louisiane,0.800000
4,Rue Stanislas Baudry,0.250000
...,...,...
122,"118, rue de Coulmiers",1.000000
123,Place Emile Zola,0.440000
124,Rue Stanislas Baudry - face rue Georges Cléme...,0.066667
125,"5, rue Racine",0.466667
